In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Imports

In [2]:
# External libraries
import os as so
import sys as s
import pathlib as pl
import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader, ConcatDataset
import torcheval
from torcheval.metrics import MulticlassF1Score, Mean
import optuna as opt
import torchvision as tn
import sklearn as sn
from sklearn.metrics import f1_score
import pandas as ps
import numpy as ny
import typing as t
import pathlib as pl
import matplotlib.pyplot as pt
import random as rng
from tqdm import tqdm
import tqdm as tm
from pprint import pprint
from git import Repo

In [3]:
# Add local package to path
if (p := pl.Path(so.getcwd(), '..').absolute().as_posix()) not in s.path:
    s.path.append(p)

# Local imports
from gic import *
from gic.tune import ResCNNHPSpace, HyperParameterSampler, Trainer
from gic.data import load_data, GenImageDataset
from gic.model import ResCNN

wandb: Currently logged in as: invokariman (castelvaar). Use `wandb login --relogin` to force relogin


/home/invokariman/Projects/git/ub-g21-deeplearning/projects/gic/notebooks/../gic/__init__.py:309: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wn_callback = WeightsAndBiasesCallback(


HyperParameter Optimization

In [4]:
# Search hyperparameters in a subspace
sampler = HyperParameterSampler[ResCNNHPSpace](lambda trial: {
    'batch_size': trial.suggest_int('batch_size', 16, 128, step=16),
    'optimizer': trial.suggest_categorical('optimizer', ['Adam']),
    'lr': trial.suggest_float('lr', 1e-4, 1e-2),
    'epochs': trial.suggest_int('epochs', 10, 50),
    'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-3),
    'pool': trial.suggest_categorical('poo', ['max', 'avg']),
    'dropout1d': trial.suggest_float('dense_dropout', 0, 0.6),
    'dropout2d': trial.suggest_float('conv_dropout', 0, 0.6),
    'conv_chan': trial.suggest_int('conv_chan', 16, 32, step=8),
    'dens_chan': trial.suggest_int('dens_chan', 128, 1024, 128),
    'activ_fn': trial.suggest_categorical('activ', ['ReLU', 'SiLU', 'GELU', 'LeakyReLU'])
})

In [ ]:
# Create the training procedure
trainer = Trainer(
    seed=SEED,
    hps=sampler,
    dataset_path=DATA_PATH,
    num_classes=CONST_NUM_CLASS,
    prefetch_factor=prefetch_factor,
    num_workers=num_workers,
    device=DEVICE
)

# Search hyperparams
cutoff = opt.pruners.HyperbandPruner()
searcher = opt.samplers.TPESampler(n_startup_trials=10)
search = opt.create_study(direction='maximize', storage=db_uri, sampler=searcher, pruner=cutoff)
search.optimize(trainer, n_trials=50, callbacks=[wn_callback])
wn.finish()

Training & Submission

In [10]:
search.best_params

{'batch_size': 96,
 'optimizer': 'Adam',
 'lr': 0.0001710922865343308,
 'epochs': 26,
 'weight_decay': 6.615022804136745e-05,
 'poo': 'max',
 'dense_dropout': 0.1509602432425139,
 'conv_dropout': 0.16877901541398593,
 'conv_chan': 32,
 'dens_chan': 640,
 'activ': 'SiLU'}

In [5]:
params = {
    'batch_size': 16,
    'optimizer': 'Adam',
    'lr': 0.0004710922865343308,
    'epochs': 30,
    'weight_decay': 6.615022804136745e-05,
    'pool': 'max',
    'dense_dropout': 0.49602432425139,
    'conv_dropout': 0.477901541398593,
    'conv_chan': 32,
    'dens_chan': 640,
    'activ': 'SiLU'
}

# Create the training procedure
trainer = Trainer(
    seed=SEED,
    hps=sampler,
    dataset_path=DATA_PATH,
    num_classes=CONST_NUM_CLASS,
    prefetch_factor=prefetch_factor,
    num_workers=num_workers,
    train_valid_split=(0.999, 0.001),
    device=DEVICE
)

# Train on `almost` the whole labeled dataset
trainer(None, t.cast(t.Any, params))
wn.finish()

epoch: 100%|██████████| 30/30 [01:34<00:00,  3.16s/it]


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train_f1_score,▁▂▃▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████
train_loss,█▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch,29
train_f1_score,0.61665
train_loss,1.23673


In [6]:
# Get model
model = trainer.model_.eval().requires_grad_(False)

# Get test data
_, _, test_data = t.cast(t.Any, load_data(DATA_PATH, True, True))
test_loader = DataLoader(test_data, batch_size, shuffle=False, prefetch_factor=prefetch_factor, num_workers=num_workers)

# Infer
preds = []
for X in test_loader:
    X = X.to(DEVICE)
    pred = torch.argmax(model(X), dim=-1)
    preds.extend(pred.cpu().tolist())

In [7]:
preds[:10]

[67, 91, 40, 13, 69, 45, 64, 54, 26, 98]

In [8]:
dataf = {'Image': [], 'Class': []}
dataf['Class'] = preds
test_data._GenImageDataset__data['Class'] = preds
test_data._GenImageDataset__data.to_csv(SUBMISSION_PATH, index=False)